**Homework 18**

**IMPORTANT!!!** Complete this notebook to do your homework, and then transfer your answers to homework18gradescope.ipynb (available on canvas).

In this assignment we'll start working with PyTorch:

In [1]:
import torch
from torch.nn import Linear, ReLU, Sequential

Let's use PyTorch to recreate the Neural Network from Homework 16:

In [2]:
network=Sequential(
    Linear(2,3),
    ReLU(),
    Linear(3,1)
)

You should be able to use it the exact same way, except that we apply PyTorch models to PyTorch tensors, rather than Numpy arrays:

In [3]:
X=torch.randn(15,2) #generate a random feature matrix with 2 features and 15 observations as a torch tensor
network(X) #Predictions of our network

tensor([[-0.2880],
        [-0.5882],
        [-0.2275],
        [-0.4799],
        [-0.7011],
        [-0.3055],
        [-0.8219],
        [-0.1546],
        [-0.3654],
        [-0.3562],
        [-0.6293],
        [-0.5050],
        [-0.4836],
        [-0.4556],
        [-0.4569]], grad_fn=<AddmmBackward0>)

You can see the weights and biases of your network as follows. Note that the first layer has a weight matrix of shape (3,2) and a bias vector of size 3, and the second layer has a weight matrix of shape(1,3) and a single bias.

In [4]:
for param in network.parameters():
    print(param,param.shape)

Parameter containing:
tensor([[-0.2574, -0.3060],
        [ 0.0158,  0.6984],
        [-0.3580,  0.0577]], requires_grad=True) torch.Size([3, 2])
Parameter containing:
tensor([-0.0503,  0.0749, -0.6119], requires_grad=True) torch.Size([3])
Parameter containing:
tensor([[ 0.3536, -0.2348,  0.4437]], requires_grad=True) torch.Size([1, 3])
Parameter containing:
tensor([-0.3654], requires_grad=True) torch.Size([1])


We now use gradient descent to train our network. Let's create a target vector:

In [5]:
target=torch.randn(15,1)

Our training loop now follows the pattern from homework 17:

In [6]:
for i in range(10000): #Do 10000 gradient descent steps
  network.zero_grad() #Zero out the derivative with respect to each network parameter

  prediction=network(X) #Compute the network prediction
  MSEloss=torch.nn.MSELoss()(prediction,target) #Compute MSE loss
  #MSEloss=((prediction-target)**2).mean() #This is the same!!

  MSEloss.backward() #Compute gradient

  for param in network.parameters():
    param.data-=0.01*param.grad  #Take a gradient descent step with learning rate of 0.01

  if i%1000==0:
    print(f"Step: {i}/10000, Loss: {MSEloss.item()}") #Periodic reporting to track progress

Step: 0/10000, Loss: 1.2329928874969482
Step: 1000/10000, Loss: 0.5991779565811157
Step: 2000/10000, Loss: 0.5090927481651306
Step: 3000/10000, Loss: 0.49895647168159485
Step: 4000/10000, Loss: 0.4952594041824341
Step: 5000/10000, Loss: 0.4950980842113495
Step: 6000/10000, Loss: 0.4949895739555359
Step: 7000/10000, Loss: 0.49499908089637756
Step: 8000/10000, Loss: 0.4950370788574219
Step: 9000/10000, Loss: 0.4950786828994751


Let's now use this on real data. We'll use the same three colunns from the `cars` dataset that we usd in Homework 7, and again use the `mpg` column for our target:

In [7]:
import pandas as pd
cars=pd.read_csv('https://vincentarelbundock.github.io/Rdatasets/csv/causaldata/auto.csv')

DWG=torch.tensor([cars.displacement,cars.weight,cars.gear_ratio],dtype=torch.float32).T
mpg=torch.tensor(cars.mpg,dtype=torch.float32)

Define a neural network appropriate for predicting `mpg` from `DWG`. Your network should have two hidden layers with 8 neurons and 4 neurons, and ReLU layers before and after all hidden layers.

In [10]:
cars_network=Sequential(
    Linear(3,8),
    ReLU(),
    Linear(8,4),
    ReLU(),
    Linear(4,1)
)

Write a training loop to train your network to predict `mpg` from `DWG`. Do 10000 gradient descent steps with a learning rate of 0.001, and report your MSE every 10000 steps.

In [14]:
lr = 0.001
report = 1000
for i in range(10000):
    cars_network.zero_grad()
    preds = cars_network(DWG)
    MSEloss = torch.nn.MSELoss()(preds,mpg)

    MSEloss.backward()

    for param in cars_network.parameters():
        param.data-=lr*param.grad

    if i%report == 0:
        print(f"Step: {i}/10000, Loss: {MSEloss.item()}") #Periodic reporting to track progress

Step: 0/10000, Loss: 33.019718170166016
Step: 1000/10000, Loss: 33.019718170166016
Step: 2000/10000, Loss: 33.019718170166016
Step: 3000/10000, Loss: 33.019718170166016
Step: 4000/10000, Loss: 33.019718170166016
Step: 5000/10000, Loss: 33.019718170166016
Step: 6000/10000, Loss: 33.019718170166016
Step: 7000/10000, Loss: 33.019718170166016
Step: 8000/10000, Loss: 33.019718170166016
Step: 9000/10000, Loss: 33.019718170166016


Compute the final MSE for your trained model:

In [17]:
final_mse= torch.nn.MSELoss()(preds,mpg).item()
final_mse

33.019718170166016

For classification problems, here are the only changes:
1. If you are predicting $n$ classes, the final layer should have $n$ neurons.
2. For training, use Categorical Cross Entropy loss (torch.nn.CrossEntropyLoss) instead of MSE. (No need to one-hot encode target variable)
3. If you want to generate predictions of your model (to guage accuracy, for example), use: `torch.argmax(network(X), dim=1)`
4. If you want to see the probability that your model predicts each class, use: `torch.softmax(network(X), dim=1)`.

With these changes in mind, we'll revisit the iris dataset:

In [24]:
from sklearn.datasets import load_iris
iris=load_iris()

X=torch.tensor(iris.data, dtype=torch.float32)
y=torch.tensor(iris.target, dtype=torch.long)

Create a neural network appropriate for predicting `y` from `X`, with one hidden layer that has 10 neurons, and ReLU layers before and after that:

In [25]:
iris_net= Sequential(
    Linear(4,10),
    ReLU(),
    Linear(10,3) # since there are 3 possible classes
)


Train your neural network for 10000 steps, with a learning rate of 0.01:

In [26]:
lr = 0.01
for i in range(10000):
    iris_net.zero_grad()
    preds = iris_net(X)
    CCEloss = torch.nn.CrossEntropyLoss()(preds,y)

    CCEloss.backward()

    for param in iris_net.parameters():
        param.data -= lr*param.grad

    if i%1000==0:
        print(f"Step: {i}/10000, Loss: {CCEloss.item()}") #Periodic reporting to track progress

Step: 0/10000, Loss: 1.3079663515090942
Step: 1000/10000, Loss: 0.20890191197395325
Step: 2000/10000, Loss: 0.1133594661951065
Step: 3000/10000, Loss: 0.0874786451458931
Step: 4000/10000, Loss: 0.07644724100828171
Step: 5000/10000, Loss: 0.07041984796524048
Step: 6000/10000, Loss: 0.06660681962966919
Step: 7000/10000, Loss: 0.06394850462675095
Step: 8000/10000, Loss: 0.06196235120296478
Step: 9000/10000, Loss: 0.060401078313589096


What is the probability that your model assigns to flower 133 being in class 1? class 2?

In [27]:
probs = torch.softmax(iris_net(X), dim=1)

In [38]:
preds = torch.argmax(probs,axis=1)
preds[preds == 1].shape[0]/preds.shape[0]

0.32666666666666666

In [45]:
class1_prob= preds[preds == 0].shape[0]/preds.shape[0]
class2_prob=preds[preds == 1].shape[0]/preds.shape[0]
class1_prob,class2_prob

(0.3333333333333333, 0.32666666666666666)

Create a vector of predictions for your model:

In [40]:
predictions= preds
predictions

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

Compute the accuracy of your model:

In [44]:
accuracy= (preds ==y).sum()/150
accuracy

tensor(0.9800)